# Uppgift 2 - Sportstatistik
- Välj 2-4 sporter och skapa lämpliga grafer/diagram för att visualisera exempelvis:
- medaljfördelning mellan länder i sporterna, åldersfördelning i sporterna
- Skapa fler plots för att visualisera olika aspekter kring sporterna.

### Våra frågeställningar:
- Kolla igenom vilka sporter som varit med?
- Analysera "Tug-Of-War"! (dragkamp) Kul att tex kolla hur länge det varit med i OS, vilka har varit bäst? Osv.
- Finns det andra gamla eller intressanta sporter att titta på?
- Alpinism?
- Aeronautics [https://www.topendsports.com/events/demonstration/aeronautics.htm](https://www.topendsports.com/events/demonstration/aeronautics.htm)
- Yngsta, äldsta medaljörerna?
#### "Tug of war" (Dragkamp) (Daniel):
- Vilka länder har deltagit och hur ser medaljfördelningen ut?
- Vilka år var detta en sport i OS?

#### Kulstötning / Slägga / Spjut / Diskus (Daniel):
- Hur ser längd- och vikt-utvecklingen ut genom historien?
- Analysera män och kvinnor separat
- Hade varit intressant att jämföra med resultaten i sporterna också....
- Distribution plots? Box-plots? Eller en animation frame, som går igenom åren?

In [ ]:
import pandas as pd
import plotly.express as px
from dash import Dash, html, dcc, callback, Output, Input
from dash_bootstrap_templates import load_figure_template
import dash_bootstrap_components as dbc
from dash import dash_table
from io import BytesIO
import base64
athlete_events = pd.read_csv("./Data/athlete_events.csv")

### 10 Bästa sporter för Frankrike

Här skapar jag en DataFrame med lämplig information för att plotta alla olika grafer som ska undersökas.

Informationen är:
- Totala medaljer vunnet av de top 10 sporterna.
- Av det totala, hur många av dem guld, silver och brons.
- Av varje guld silver och brons. Hur många var kvinnor och hur många var män.

In [ ]:
only_france_df = athlete_events[athlete_events['Team'] == 'France']
only_france_df.head()
medal_counts = only_france_df.groupby('Sport')['Medal'].value_counts().unstack().fillna(0)
medal_counts[['Gold', 'Silver', 'Bronze']]
medal_counts['Total'] = medal_counts[['Gold', 'Silver', 'Bronze']].sum(axis=1)
medal_counts[['Gold', 'Silver', 'Bronze', 'Total']]
medal_counts['Participations'] = medal_counts.sum(axis=1)
medal_counts[['Gold', 'Silver', 'Bronze', 'Total', 'Participations']]
gold_female_counts = only_france_df[only_france_df['Medal'] == 'Gold'].loc[only_france_df['Sex'] == 'F'].groupby('Sport').size()
medal_counts['Gold Female'] = gold_female_counts.fillna(0)
silver_female_counts = only_france_df[only_france_df['Medal'] == 'Silver'].loc[only_france_df['Sex'] == 'F'].groupby('Sport').size()
medal_counts['Silver Female'] = silver_female_counts.fillna(0)

bronze_female_counts = only_france_df[only_france_df['Medal'] == 'Bronze'].loc[only_france_df['Sex'] == 'F'].groupby('Sport').size()
medal_counts['Bronze Female'] = bronze_female_counts.fillna(0)

gold_male_counts = only_france_df[only_france_df['Medal'] == 'Gold'].loc[only_france_df['Sex'] == 'M'].groupby('Sport').size()
medal_counts['Gold Male'] = gold_male_counts.fillna(0)

silver_male_counts = only_france_df[only_france_df['Medal'] == 'Silver'].loc[only_france_df['Sex'] == 'M'].groupby('Sport').size()
medal_counts['Silver Male'] = silver_male_counts.fillna(0)

bronze_male_counts = only_france_df[only_france_df['Medal'] == 'Bronze'].loc[only_france_df['Sex'] == 'M'].groupby('Sport').size()
medal_counts['Bronze Male'] = bronze_male_counts.fillna(0)

medal_counts['Total Female'] = medal_counts[['Gold Female', 'Silver Female', 'Bronze Female']].sum(axis=1)
medal_counts['Total Male'] = medal_counts[['Gold Male', 'Silver Male', 'Bronze Male']].sum(axis=1)
medal_counts.fillna(0, inplace=True)

medal_counts.head(10)

In [ ]:
top_10_sports_total = medal_counts.nlargest(10, 'Total')
top_10_sports_total_plot = px.bar(top_10_sports_total, x=top_10_sports_total.index, y=['Bronze', 'Silver', 'Gold'], title='Top 10 sports with the highest total medals won by France', barmode='stack')

top_10_sports_total_plot.update_layout(
    paper_bgcolor="black",
    plot_bgcolor="black",
    font_color="white",
    yaxis_title="Total medals won",
)
top_10_sports_total_plot.update_traces(marker_color='#CD7F32', selector=dict(name='Bronze'))
top_10_sports_total_plot.update_traces(marker_color='silver', selector=dict(name='Silver'))
top_10_sports_total_plot.update_traces(marker_color='gold', selector=dict(name='Gold'))
top_10_sports_total_plot.show()

In [ ]:
top_10_sports_gold = medal_counts.nlargest(10, 'Gold')
top_10_sports_gold_plot = px.bar(top_10_sports_gold, x=top_10_sports_gold.index, y=['Gold Female', 'Gold Male'], title='Top 10 sports with the highest number of gold medals won by France', barmode='stack')
top_10_sports_gold_plot.update_layout(
    paper_bgcolor="black",
    plot_bgcolor="black",
    font_color="white",
    yaxis_title="Number of gold medals won"
)
top_10_sports_gold_plot.show()

In [ ]:
top_10_sports_silver = medal_counts.nlargest(10, 'Silver')
top_10_sports_silver_plot = px.bar(top_10_sports_silver, x=top_10_sports_silver.index, y=['Silver Female', 'Silver Male'], title='Top 10 sports with the highest number of silver medals won by France', barmode='stack')
top_10_sports_silver_plot.update_layout(
    paper_bgcolor="black",
    plot_bgcolor="black",
    font_color="white",
    yaxis_title="Number of silver medals won"
)
top_10_sports_silver_plot.show()

In [ ]:
top_10_sports_bronze = medal_counts.nlargest(10, 'Bronze')
top_10_sports_bronze_plot = px.bar(top_10_sports_bronze, x=top_10_sports_bronze.index, y=['Bronze Female', 'Bronze Male'], title='Top 10 sports with the highest number of bronze medals won by France', barmode='stack')
top_10_sports_bronze_plot.update_layout(
    paper_bgcolor="black",
    plot_bgcolor="black",
    font_color="white",
    yaxis_title="Number of bronze medals won"
)
top_10_sports_bronze_plot.show()

### Histogram över olika åldrar över tid

In [ ]:
# Filter the dataframe to include only rows where the Team is 'France'
only_france_df = athlete_events[athlete_events['Team'] == 'France']
only_france_df.sort_values(by=['Year'], inplace=True)
only_france_df.dropna(subset=['Age'], inplace=True)

# Create a histogram of ages over time
histogram_ages_over_time = px.histogram(only_france_df, x='Age', title='Histogram of Ages Over Time', animation_frame='Year', range_x=[10, 80])

# Customize the layout of the histogram
histogram_ages_over_time.update_layout(
    xaxis_title='Age',
    yaxis_title='Count',
    paper_bgcolor="black",
    plot_bgcolor="black",
    font_color="white",
    yaxis_range=[0, 100]
)

# Show the histogram
histogram_ages_over_time.show()

In [ ]:
fencing_df = only_france_df[only_france_df['Sport'] == 'Fencing']



# FRA    (Ars)

In [ ]:
import hashlib as hl
import pandas as pd
import plotly.express as px
athlete_events = pd.read_csv("./Data/athlete_events.csv")

In [ ]:
filter_france = athlete_events["NOC"] == "FRA" 
france = athlete_events[filter_france] # New DF france 
france.reset_index(drop=True, inplace=True) # set new idx
# Anonym DF of FRA
france.loc[:, "Name"] = france.loc[:, "Name"].apply(lambda name: hl.sha256(name.encode()).hexdigest()) 
display(france.shape, france.head(1))

In [ ]:
all_year_medals = france.groupby("Year")["Medal"].value_counts().unstack().fillna(0)
all_year_medals.reset_index(inplace=True)
all_year_medals.head(3)

In [ ]:
plt_all_years_medals = px.bar(all_year_medals, x="Year", y=["Gold", "Silver", "Bronze"],
                              barmode="stack", 
                              title="Frankrike: OS",
                              color_discrete_sequence=['#FFD700', '#C0C0C0', '#CD7F32'])
plt_all_years_medals.update_layout(legend_title_text="Medalj", 
                                yaxis_title="Antal",
                                xaxis_title="År")
plt_all_years_medals.update_traces(hovertemplate='Medaljer: %{y}<br>År: %{x}')
plt_all_years_medals.show()

In [ ]:
# Summer
filter_winter = france["Season"] == "Summer" 
summer_fra = france[filter_winter] # New DF only summer
# Winter
filter_winter = france["Season"] == "Winter" 
winter_fra = france[filter_winter] # New DF only summer

In [ ]:
summer_medals = summer_fra.groupby("Year")["Medal"].value_counts().unstack().fillna(0)
summer_medals.reset_index(inplace=True)
summer_medals.head(1)

In [ ]:
plt_summer_medals = px.bar(summer_medals, x="Year", y=["Gold", "Silver", "Bronze"],
                            barmode="stack",
                            title="Frankrike: Sommar OS",
                            color_discrete_sequence=['#FFD700', '#C0C0C0', '#CD7F32'])
plt_summer_medals.update_layout(legend_title_text="Medalj", 
                                yaxis_title="Antal",
                                xaxis_title="År")
plt_summer_medals.update_traces(hovertemplate='Medaljer: %{y}<br>År: %{x}')
plt_summer_medals.show()

In [ ]:
winter_medals = winter_fra.groupby("Year")["Medal"].value_counts().unstack().fillna(0)
winter_medals.reset_index(inplace=True)
winter_medals.head(1)

In [ ]:
plt_winter_medals = px.bar(winter_medals, x="Year", y=["Gold", "Silver", "Bronze"],
                            barmode="stack",
                            title="Frankrike: Vinter OS",
                            color_discrete_sequence=['#FFD700', '#C0C0C0', '#CD7F32'])
plt_winter_medals.update_layout(legend_title_text="Medalj", 
                                yaxis_title="Antal",
                                xaxis_title="År")
plt_winter_medals.update_traces(hovertemplate='Medaljer: %{y}<br>År: %{x}')
plt_winter_medals.show()

In [ ]:
all_medals = athlete_events.groupby("Year")["Medal"].count()
all_medals = all_medals.reset_index()
all_medals.head(2)

In [ ]:
fra_all_medals = france.groupby("Year")["Medal"].count()
fra_all_medals = fra_all_medals.reset_index()
fra_all_medals.head(2)

In [ ]:
fra_all_medals["Andel_%"] = (fra_all_medals.loc[:, "Medal"] / all_medals.loc[:, "Medal"]) * 100
fra_all_medals.head(1)

In [ ]:
plt_fra_andel = px.line(fra_all_medals, x="Year", y="Andel_%", title="Frankrike: Andel medaljer per OS")
plt_fra_andel.update_layout(yaxis_title="Andel - %", xaxis_title="År")
plt_fra_andel.show()

# Weightlifting

In [ ]:
# Create DataFrame with only sport = weightlifting
weightlifting = athlete_events.query("Sport == 'Weightlifting'") # DF med all weightlifting

#remove NaN's from weight, height and age:
weightlifting = weightlifting.dropna(subset='Weight')
weightlifting = weightlifting.dropna(subset='Height')
weightlifting = weightlifting.dropna(subset='Age')

# removes "weightlifting" from Event column:
weightlifting['Event'] = weightlifting['Event'].str.replace("Weightlifting ","")

display(weightlifting['Event'].unique()) # kollar vilka event som finns
px.box(weightlifting.query('''Event == "Women's Super-Heavyweight"'''), x='Year', y='Weight')
#display(type(weightlifting))

# Speed skating

In [ ]:
x = athlete_events['Sport'].str.contains('Speed Skating')
speedskating = athlete_events.loc[x]
speedskating['Event'].unique()

In [ ]:
# filtrerar dataframen så att bara de events vi är intresserade av kommer med:
speed_skating = athlete_events[
    (athlete_events['Event'] == "Speed Skating Men's 1,500 metres") |
    (athlete_events['Event'] == "Speed Skating Men's 500 metres") |
    (athlete_events['Event'] == "Speed Skating Men's 5,000 metres") |
    (athlete_events['Event'] == "Speed Skating Men's 10,000 metres") |
    (athlete_events['Event'] == "Speed Skating Women's 1,000 metres") |
    (athlete_events['Event'] == "Speed Skating Women's 1,500 metres") |
    (athlete_events['Event'] == "Speed Skating Women's 500 metres") |
    (athlete_events['Event'] == "Speed Skating Men's 1,000 metres") |
    (athlete_events['Event'] == "Speed Skating Women's 3,000 metres") |
    (athlete_events['Event'] == "Speed Skating Women's 5,000 metres")
    ]
speed_skating

In [ ]:
# dropna NaN's in medal column:
speed_skating = speed_skating.dropna(axis=0, subset='Medal')
speed_skating

# remove "Speed Skating" som är onödig:
speed_skating['Event'] = speed_skating['Event'].str.replace("Speed Skating ","")

In [ ]:
speed_skating_gb_df = speed_skating.groupby(['Team','Medal','Event','Year'])['Medal'].count().reset_index(name='Medal Count')

# # sorterar ut  de 10 bästa länderna map antal guldmedaljer
# speed_sakting_sorted = speed_skating_gb_df[speed_skating_gb_df['Medal'] == 'Gold'].groupby('Team')['Medal Count'].sum().sort_values(ascending=False).reset_index()
# top_10_countries = speed_sakting_sorted.head(10)['Team'].tolist()
# speed_skating_df_top_10 = speed_skating_gb_df[speed_skating_gb_df['Team'].isin(top_10_countries)]
# speed_skating_df_top_10['Team'].sort_values()
# display(speed_skating_df_top_10.head())

# Mask out years with Speed Skating
years = speed_skating_gb_df['Year'].sort_values().unique().tolist()
display(years)

In [ ]:
app = Dash(
    __name__,
    suppress_callback_exceptions=True,
    external_stylesheets=[dbc.themes.FLATLY],
    meta_tags=[{"name": "viewport", "content": "width=device-width, initial-scale=1"}]
)

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.H1("Speed Skating Dashboard", className='text-center text-primary mx-3'),
            html.Br(),
            dcc.Dropdown(
                id="multi_dropdown",
                multi=True,
                searchable=False, 
                className='mb-2',
                placeholder='Select event',
                options=[event for event in speed_skating_gb_df['Event'].unique()],
                value=["Men's 1,500 metres"]
            ),
            html.Br(),
            html.P("Select year interval:"),
            dcc.RangeSlider(
                min=min(years),
                max=max(years),
                step=1,
                marks=None,
                id="RangeSlider",
                ),
            dcc.Graph(
                id='skate_graph',
                config={'staticPlot':True},
                figure={}
            ),
            html.Button("Download Image", id="download-img-btn", n_clicks=0),
            dcc.Download(id="img-downloader"),
            html.Br(),
            html.H1("Weightlifting: Age, Weight and Height Analysis over time."),
            html.P("Select class for analysis:"),
            dcc.RadioItems(
                options=[event for event in weightlifting['Event'].unique()],
                id="weightlifting-event",
                value="Women's Super-Heavyweight"
            ),
            html.P("Select analysis:"),
            dcc.RadioItems(
                ['Age','Weight','Height'],
                id="weightlifting-attr",
                value='Age'
            ),
            dcc.Graph(
                id='weightlifting-graph',
                config={'staticPlot':True},
                figure={}
            )
        ], xs=12, sm=11, md=10, lg=5)
    ], justify='center'),
], fluid=True)


@callback(
    Output("skate_graph", "figure"),
    Input("RangeSlider", "value"),
    Input("multi_dropdown", "value")
)

def update_multidropdown(year, events):
    if events is None: events = []
    if year is None: year = [1896, 1922]
    df = speed_skating_gb_df[(speed_skating_gb_df['Event'].isin(events)) & (speed_skating_gb_df['Year'] >= year[0]) & (speed_skating_gb_df['Year'] <= year[1])]
    return px.bar(df,
                x='Team',
                y='Medal Count',
                hover_name='Event',
                hover_data='Event',
                color='Medal',
                title=f'Speed Skating medals, between {year[0]} and {year[1]}:')

@callback(
    Output('img-downloader', 'data'),
    Input('download-img-btn', 'n_clicks' ),
    Input('skate-graph','figure')
)
def save_img(n_clicks, figure):
    if n_clicks:
        img_bytes = figure.to_image(format='png')
        return dcc.send_bytes(img_bytes, "../Images/speed_skate.png")
        
@callback(
    Output('weightlifting-graph','figure'),
    Input('weightlifting-event','value'),
    Input('weightlifting-attr', 'value')
)

def update_weightlifting(event, attr):
    event = [event] #måste göra om str till list
    if event is None: event = ["Women's Super-Heavyweight"]
    if attr is None: attr = []
    df = weightlifting[(weightlifting['Event'].isin(event))]
    return px.box(
        df,
        x='Year',
        y=attr,
        title=f'{attr} for {event[0]}.')

     
# app.run(host='0.0.0.0', port=8050,debug=True, jupyter_mode='external')
app.run(debug=True, jupyter_mode='external')

# ARS

In [ ]:
filter_basket = athlete_events["Sport"] == "Basketball"
basket = athlete_events[filter_basket]
basket.head(1)

In [ ]:
top_basket = basket.groupby("NOC")["Medal"].value_counts().unstack().fillna(0)
top_basket.head(1)

In [ ]:
top_basket["Tot"] = top_basket.loc[:, "Bronze"] + top_basket.loc[:, "Silver"] + top_basket.loc[:, "Gold"] 
top_basket.sort_values(by="Tot", ascending=False, inplace=True)
top_basket.reset_index(inplace=True)
top_ten_basket = top_basket.loc[:9, :]
top_ten_basket.head(1)

In [ ]:
plt_top_basket = px.bar(top_ten_basket, x="NOC", y=["Gold", "Silver", "Bronze"],
                        barmode="stack", 
                        title="Top 10 Basketball",
                        color_discrete_sequence=['#FFD700', '#C0C0C0', '#CD7F32'])
plt_top_basket.update_layout(legend_title_text="Medal", 
                                yaxis_title="Count",
                                xaxis_title="Countries")
plt_top_basket.show()

# Wrestling

In [ ]:
filter_wrest = athlete_events["Sport"] == "Wrestling"
wrestling = athlete_events[filter_wrest]
wrestling.head(1)

In [ ]:
top_wrestling = wrestling.groupby("NOC")["Medal"].value_counts().unstack().fillna(0)
top_wrestling.head(1)

In [ ]:
top_wrestling["Tot"] = top_wrestling.loc[:, "Bronze"] + top_wrestling.loc[:, "Silver"] + top_wrestling.loc[:, "Gold"] 
top_wrestling.sort_values(by="Tot", ascending=False, inplace=True)
top_wrestling.reset_index(inplace=True)
top_ten_wrestling = top_wrestling.loc[:9, :]
top_ten_wrestling

In [ ]:
plt_top_wrestling = px.bar(top_ten_wrestling, x="NOC", y=["Gold", "Silver", "Bronze"],
                        barmode="stack", 
                        title="Top 10 Wrestling",
                        color_discrete_sequence=['#FFD700', '#C0C0C0', '#CD7F32'])
plt_top_wrestling.update_layout(legend_title_text="Medal", 
                                yaxis_title="Count",
                                xaxis_title="Countries")
plt_top_wrestling.show()

# Height

In [ ]:
avg_height_wrestling = wrestling.groupby("NOC")["Height"].mean().reset_index()
avg_height_wrestling = avg_height_wrestling.dropna(subset=['Height'])
avg_height_wrestling.head(2)

avg_height_basket = basket.groupby("NOC")["Height"].mean().reset_index()
avg_height_basket = avg_height_basket.dropna(subset=['Height'])
avg_height_basket.head(5)

In [ ]:
merged_height = pd.merge(avg_height_wrestling, avg_height_basket, on="NOC", suffixes=(' Wrestling', ' Basketball'))
display(merged_height.shape, merged_height.head(3))

In [ ]:
plt_height_compair = px.line(merged_height, x="NOC", y=["Height Basketball", "Height Wrestling"],
                             title="Average Height: Basketball vs Wrestling")
plt_height_compair.update_layout(legend_title_text="Sports",
                                yaxis_title="Length cm",
                                xaxis_title="Countries")
plt_height_compair.show()

In [ ]:
b = merged_height["Height Basketball"].mean()
w = merged_height["Height Wrestling"].mean()
print(b, w)

# Gender Distribution

In [ ]:
all_pep_basket = basket["Name"].nunique()
all_pep_wrestling = wrestling["Name"].nunique()

# W wrest
women_w = wrestling[wrestling["Sex"] == "F"]
women_wrestling = women_w["Name"].nunique()
men_wrestling = all_pep_wrestling - women_wrestling

# W basket
women_b = basket[basket["Sex"] == "F"]
women_basket = women_b["Name"].nunique()
men_basket = all_pep_basket - women_basket


In [ ]:
my_df = {'Sports': ['Basket', 'Wrestling'],
               'M': [men_basket, men_wrestling],
               'F': [women_basket, women_wrestling],
               'All': [all_pep_basket, all_pep_wrestling],
               'Women %': [(women_basket/all_pep_basket)*100, (women_wrestling/all_pep_wrestling)*100],
               'Men %': [(men_basket/all_pep_basket)*100, (men_wrestling/all_pep_wrestling)*100]}
gender_df = pd.DataFrame(my_df)
gender_df


In [ ]:
plt_gender = px.bar(gender_df, x="Sports", y=["Men %", "Women %"],
                    title="Gender Distribution",
                    barmode="group",
                    color_discrete_sequence=['#383fff', '#ff8af5'])
plt_gender.update_layout(legend_title_text="Sex", 
                                yaxis_title="%")
plt_gender.show()

# Dash

In [ ]:
from dash import Dash, html, dcc, callback, Output, Input, State
from dash_bootstrap_templates import load_figure_template
import dash_bootstrap_components as dbc
from dash import dash_table

In [ ]:
# load_figure_template("flatly")

# app = Dash(
#     __name__, 
#     external_stylesheets=[dbc.themes.FLATLY],
#     meta_tags=[{"name": "viewport", "content": "width=device-width, initial-scale=1"}]
# )

# app.layout = dbc.Container([
#     dbc.Row([
#         dbc.Col([
#             html.H1("Uppgift_2", className='text-center text-primary mx-3'),
#             html.Br()
#         ], width=12, style={'text-align': 'center'})
#     ]),

#     dbc.Row([
#         dbc.Col([
#             dcc.Graph(id="basket", figure=plt_top_basket)
#         ], xs=12, sm=11, md=10, lg=8),
#         dbc.Col([
#             dcc.Graph(id="wrestling", figure=plt_top_wrestling)
#         ], xs=12, sm=11, md=10, lg=8),
#         ], justify="evenly"),
    
#     dbc.Row([
#         dbc.Col([
#             html.Br(),
#             html.H2('Height: Basket vs Wrestling', id='height_vs'),
#             dash_table.DataTable(id='title_height', page_size=15)
#         ], width=10),
#         dbc.Col([
#             dcc.Graph(id="height", figure=plt_height_compair)
#         ], xs=12, sm=11, md=10, lg=8),
#     ], justify='center'),

#     dbc.Row([
#         dbc.Col([
#             html.Br(),
#             html.H2('Gender Distribution', id='data_label'),
#             dash_table.DataTable(id='title_gen', page_size=15)
#         ], width=10),
#         dbc.Col([
#             dcc.Graph(id="gender", figure=plt_gender)
#         ], xs=12, sm=11, md=10, lg=8),
#     ], justify='center'),
# ], fluid=True)

# app.run(debug=True, jupyter_mode='external')

In [ ]:
load_figure_template("flatly")

app = Dash(
    __name__, 
    external_stylesheets=[dbc.themes.FLATLY],
    meta_tags=[{"name": "viewport", "content": "width=device-width, initial-scale=1"}]
)

# Antag att plt_top_basket och plt_top_wrestling redan är definierade plotly grafer

# Skapa en lista med alternativ för dropdown-menyn
options = [
    {'label': 'Top Basket', 'value': 'basket'},
    {'label': 'Top Wrestling', 'value': 'wrestling'}
]

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.H1("Uppgift_2", className='text-center text-primary mx-3'),
            html.Br()
        ], width=12, style={'text-align': 'center'})
    ]),
    
    # Lägg till en Dropdown för att välja mellan graferna
    dbc.Row([
        dbc.Col([
            dcc.Dropdown(
                id='graph-dropdown',
                options=options,
                value='basket'  # Standardvärde för grafen
            )
        ], xs=12, sm=11, md=10, lg=8),
    ], justify="center"),

    # Lägg till en plats för att visa vald graf
    dbc.Row([
        dbc.Col([
            dcc.Graph(id="selected_graph")
        ], xs=12, sm=11, md=10, lg=8),
    ], justify="center"),
    
    # Resten av din layout här

], fluid=True)


# Skapa en callback-funktion för att uppdatera den valda grafen baserat på dropdown-valet
@app.callback(
    Output('selected_graph', 'figure'),
    [Input('graph-dropdown', 'value')]
)
def update_graph(selected_value):
    # Returnera rätt figur baserat på dropdown-valet
    if selected_value == 'basket':
        return plt_top_basket
    elif selected_value == 'wrestling':
        return plt_top_wrestling

# Starta servern
app.run_server(debug=True, jupyter_mode='external')
